In [1]:
import numpy as np
import pickle
import os

seed = 2023

In [2]:
import torch

# set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [3]:
from nlpsig_networks.scripts.seqsignet_attention_encoder_functions import (
    seqsignet_attention_encoder_hyperparameter_search,
)

In [4]:
output_dir = "client_talk_type_output"
if not os.path.isdir(output_dir):
    os.makedirs(output_dir)

## AnnoMI

In [5]:
%run ../load_anno_mi.py

In [6]:
anno_mi.head()

,mi_quality,transcript_id,topic,utterance_id,interlocutor,timestamp,utterance_text,annotator_id,therapist_input_exists,therapist_input_subtype,reflection_exists,reflection_subtype,question_exists,question_subtype,main_therapist_behaviour,client_talk_type,datetime,speaker
0,high,0,reducing alcohol consumption,0,therapist,00:00:13,Thanks for filling it out. We give this form t...,3,False,NaN,False,NaN,True,open,question,NaN,2023-11-06 00:00:13,-1
1,high,0,reducing alcohol consumption,1,client,00:00:24,Sure.,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neutral,2023-11-06 00:00:24,1
2,high,0,reducing alcohol consumption,2,therapist,00:00:25,"So, let's see. It looks that you put-- You dri...",3,True,information,False,NaN,False,NaN,therapist_input,NaN,2023-11-06 00:00:25,-1
3,high,0,reducing alcohol consumption,3,client,00:00:34,Mm-hmm.,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neutral,2023-11-06 00:00:34,1
4,high,0,reducing alcohol consumption,4,therapist,00:00:34,-and you usually have three to four drinks whe...,3,True,information,False,NaN,False,NaN,therapist_input,NaN,2023-11-06 00:00:34,-1


In [7]:
with open("../anno_mi_sbert.pkl", "rb") as f:
    sbert_embeddings = pickle.load(f)

sbert_embeddings.shape

(9699, 384)

In [8]:
features = ["time_encoding", "timeline_index"]
standardise_method = ["z_score", None]
include_features_in_path = True
include_features_in_input = True

In [9]:
num_epochs = 100
dimensions = [15]  # [50, 15]
swmhau_parameters = [(12, 3, 10), (8, 4, 6)]#, (8, 4, 12)]
num_layers = [1]
ffn_hidden_dim_sizes = [[256,256],[512,512]]
dropout_rates = [0.1, 0.2]
learning_rates = [1e-3, 1e-4, 5e-4]
seeds = [1, 12, 123]
loss = "focal"
gamma = 2
validation_metric = "f1"
patience = 5

In [10]:
kwargs = {
    "num_epochs": num_epochs,
    "df": anno_mi,
    "id_column": "transcript_id",
    "label_column": "client_talk_type",
    "embeddings": sbert_embeddings,
    "y_data": y_data_client,
    "output_dim": output_dim_client,
    "dimensions": dimensions,
    "log_signature": True,
    "pooling": "signature",
    "transformer_encoder_layers": 2,
    "swmhau_parameters": swmhau_parameters,
    "num_layers": num_layers,
    "ffn_hidden_dim_sizes": ffn_hidden_dim_sizes,
    "dropout_rates": dropout_rates,
    "learning_rates": learning_rates,
    "seeds": seeds,
    "loss": loss,
    "gamma": gamma,
    "device": device,
    "features": features,
    "standardise_method": standardise_method,
    "include_features_in_path": include_features_in_path,
    "include_features_in_input": include_features_in_input,
    "path_indices": client_index,
    "split_ids": client_transcript_id,
    "k_fold": True,
    "patience": patience,
    "validation_metric": validation_metric,
    "verbose": False,
}

# history_length=11

In [11]:
shift = 3
window_size = 5
n = 3

In [12]:
(
    seqsignet_attention_encoder_umap_kfold_11,
    best_seqsignet_attention_encoder_umap_kfold_11,
    _,
    __,
) = seqsignet_attention_encoder_hyperparameter_search(
    shift=shift,
    window_size=window_size,
    n=n,
    dim_reduce_methods=["umap"],
    results_output=f"{output_dir}/seqsignet_attention_encoder_umap_focal_{gamma}_{shift}_{window_size}_{n}_kfold.csv",
    **kwargs,
)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


##################################################
dimension: 15 | method: umap
given shift 3, window size 5 and n 3: history length = 11
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.


...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

given shift 3, window size 5 and n 3: history length = 11
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.
saving results dataframe to CSV for this hyperparameter search in client_talk_type_output/seqsignet_attention_encoder_umap_focal_2_3_5_3_kfold.csv
saving the best model results dataframe to CSV for this hyperparameter search in client_talk_type_output/seqsignet_attention_encoder_umap_focal_2_3_5_3_kfold_best_model.csv


In [13]:
seqsignet_attention_encoder_umap_kfold_11

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,seed,loss_function,gamma,k_fold,n_splits,augmentation_type,hidden_dim_aug,comb_method,batch_size,model_id
0,None,0.636207,0.517305,"[0.7609110947832475, 0.4221453287197232, 0.368...",0.510939,"[0.77352853301464, 0.44363636363636366, 0.3156...",0.531658,"[0.7486986697513013, 0.40264026402640263, 0.44...",None,0.668433,...,1,focal,2,True,5,Conv1d,None,concatenation,64,0
0,None,0.652299,0.516845,"[0.7754170825609582, 0.42781222320637735, 0.34...",0.518182,"[0.7648382559774964, 0.46175908221797324, 0.32...",0.517966,"[0.7862926547137074, 0.39851485148514854, 0.36...",None,0.678709,...,12,focal,2,True,5,Conv1d,None,concatenation,64,0
0,None,0.627586,0.488205,"[0.7584423049288691, 0.3978590544157003, 0.308...",0.488003,"[0.7537846329620108, 0.4330097087378641, 0.277...",0.492806,"[0.7631578947368421, 0.367986798679868, 0.3472...",None,0.660405,...,123,focal,2,True,5,Conv1d,None,concatenation,64,0
0,None,0.633525,0.456989,"[0.7681241184767279, 0.39461172741679873, 0.20...",0.480297,"[0.7497246696035242, 0.3795731707317073, 0.311...",0.451568,"[0.7874493927125507, 0.41089108910891087, 0.15...",None,0.640334,...,1,focal,2,True,5,Conv1d,None,concatenation,64,1
0,None,0.610345,0.435463,"[0.7555053642010163, 0.34503171247357295, 0.20...",0.441052,"[0.7380033094318809, 0.3538594969644406, 0.231...",0.431982,"[0.7738577212261423, 0.33663366336633666, 0.18...",None,0.627007,...,12,focal,2,True,5,Conv1d,None,concatenation,64,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,None,0.654406,0.337919,"[0.7927298988926337, 0.12396694214876032, 0.09...",0.435938,"[0.6789690721649485, 0.375, 0.25384615384615383]",0.362181,"[0.9522845575477155, 0.07425742574257425, 0.06]",None,0.644188,...,12,focal,2,True,5,Conv1d,None,concatenation,64,22
0,None,0.652490,0.401146,"[0.784103935804356, 0.30220356768100737, 0.117...",0.472133,"[0.7006601411336216, 0.414985590778098, 0.3007...",0.400154,"[0.8901098901098901, 0.2376237623762376, 0.072...",None,0.646435,...,123,focal,2,True,5,Conv1d,None,concatenation,64,22
0,None,0.639847,0.497814,"[0.7685854787387908, 0.4260692464358452, 0.298...",0.498889,"[0.7688078703703703, 0.42075623491552694, 0.30...",0.496930,"[0.7683632157316368, 0.4315181518151815, 0.290...",None,0.659281,...,1,focal,2,True,5,Conv1d,None,concatenation,64,23
0,None,0.641762,0.491691,"[0.7743954480796585, 0.39695302581464237, 0.30...",0.496476,"[0.7620380739081747, 0.40747176368375326, 0.31...",0.487738,"[0.7871602082128398, 0.38696369636963696, 0.28...",None,0.667148,...,12,focal,2,True,5,Conv1d,None,concatenation,64,23


In [14]:
best_seqsignet_attention_encoder_umap_kfold_11["f1"].mean()

0.5236556839152277

In [15]:
best_seqsignet_attention_encoder_umap_kfold_11["precision"].mean()

0.5202464466883728

In [16]:
best_seqsignet_attention_encoder_umap_kfold_11["recall"].mean()

0.5313548570974314

In [17]:
np.stack(best_seqsignet_attention_encoder_umap_kfold_11["f1_scores"]).mean(axis=0)

array([0.77424456, 0.43769494, 0.35902755])

In [18]:
np.stack(best_seqsignet_attention_encoder_umap_kfold_11["precision_scores"]).mean(
    axis=0
)

array([0.77754646, 0.46120103, 0.32199185])

In [19]:
np.stack(best_seqsignet_attention_encoder_umap_kfold_11["recall_scores"]).mean(axis=0)

array([0.77106227, 0.41694169, 0.40606061])

# history_length=20

In [20]:
shift = 3
window_size = 5
n = 6

In [21]:
(
    seqsignet_attention_encoder_umap_kfold_20,
    best_seqsignet_attention_encoder_umap_kfold_20,
    _,
    __,
) = seqsignet_attention_encoder_hyperparameter_search(
    shift=shift,
    window_size=window_size,
    n=n,
    dim_reduce_methods=["umap"],
    results_output=f"{output_dir}/seqsignet_attention_encoder_umap_focal_{gamma}_{shift}_{window_size}_{n}_kfold.csv",
    **kwargs,
)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


##################################################
dimension: 15 | method: umap
given shift 3, window size 5 and n 6: history length = 20
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.


...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

given shift 3, window size 5 and n 6: history length = 20
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.
saving results dataframe to CSV for this hyperparameter search in client_talk_type_output/seqsignet_attention_encoder_umap_focal_2_3_5_6_kfold.csv
saving the best model results dataframe to CSV for this hyperparameter search in client_talk_type_output/seqsignet_attention_encoder_umap_focal_2_3_5_6_kfold_best_model.csv


In [22]:
seqsignet_attention_encoder_umap_kfold_20

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,seed,loss_function,gamma,k_fold,n_splits,augmentation_type,hidden_dim_aug,comb_method,batch_size,model_id
0,None,0.643103,0.520138,"[0.7707562527749, 0.44472681067344344, 0.34493...",0.513862,"[0.7893301000303122, 0.45691906005221933, 0.29...",0.533583,"[0.7530364372469636, 0.43316831683168316, 0.41...",None,0.667630,...,1,focal,2,True,5,Conv1d,None,concatenation,64,0
0,None,0.642529,0.524507,"[0.7667409057164067, 0.4487072560467056, 0.358...",0.516885,"[0.7879157766249618, 0.4536256323777403, 0.309...",0.538674,"[0.7466743782533256, 0.4438943894389439, 0.425...",None,0.670681,...,12,focal,2,True,5,Conv1d,None,concatenation,64,0
0,None,0.652874,0.526581,"[0.7806940799066783, 0.4381560931081698, 0.360...",0.525368,"[0.7873529411764706, 0.4902962206332993, 0.298...",0.542183,"[0.7741469057258531, 0.39603960396039606, 0.45...",None,0.676622,...,123,focal,2,True,5,Conv1d,None,concatenation,64,0
0,None,0.615900,0.472097,"[0.7499252168710738, 0.43003663003663, 0.23632...",0.472870,"[0.7766418835192069, 0.38669301712779974, 0.25...",0.476436,"[0.7249855407750144, 0.48432343234323433, 0.22]",None,0.637765,...,1,focal,2,True,5,Conv1d,None,concatenation,64,1
0,None,0.620690,0.436597,"[0.7639674437853498, 0.32870577698163905, 0.21...",0.448256,"[0.7304141387496703, 0.3594515181194907, 0.254...",0.430883,"[0.8007518796992481, 0.3028052805280528, 0.189...",None,0.641779,...,12,focal,2,True,5,Conv1d,None,concatenation,64,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,None,0.657854,0.404757,"[0.7887751232461131, 0.28879310344827586, 0.13...",0.496928,"[0.7006512463507748, 0.4161490683229814, 0.373...",0.402338,"[0.9022556390977443, 0.22112211221122113, 0.08...",None,0.653340,...,12,focal,2,True,5,Conv1d,None,concatenation,64,22
0,None,0.664176,0.313107,"[0.7953658824920203, 0.1439552760307477, 0.0]",0.380995,"[0.6726654669066187, 0.4703196347031963, 0.0]",0.352600,"[0.9728166570271833, 0.08498349834983498, 0.0]",None,0.643224,...,123,focal,2,True,5,Conv1d,None,concatenation,64,22
0,None,0.651533,0.509709,"[0.7789896670493685, 0.42976588628762546, 0.32...",0.511743,"[0.7732193732193732, 0.43559322033898307, 0.32...",0.507828,"[0.7848467322151532, 0.4240924092409241, 0.314...",None,0.662653,...,1,focal,2,True,5,Conv1d,None,concatenation,64,23
0,None,0.646552,0.482977,"[0.7812630698452531, 0.38458008260670035, 0.28...",0.491261,"[0.7542395693135936, 0.4332988624612203, 0.286...",0.478668,"[0.810294968189705, 0.3457095709570957, 0.28]",None,0.668112,...,12,focal,2,True,5,Conv1d,None,concatenation,64,23


In [23]:
best_seqsignet_attention_encoder_umap_kfold_20["f1"].mean()

0.525800702456729

In [24]:
best_seqsignet_attention_encoder_umap_kfold_20["precision"].mean()

0.5223519601756575

In [25]:
best_seqsignet_attention_encoder_umap_kfold_20["recall"].mean()

0.5344082661591781

In [26]:
np.stack(best_seqsignet_attention_encoder_umap_kfold_20["f1_scores"]).mean(axis=0)

array([0.77664131, 0.44586663, 0.35489417])

In [27]:
np.stack(best_seqsignet_attention_encoder_umap_kfold_20["precision_scores"]).mean(
    axis=0
)

array([0.78199726, 0.47006374, 0.31499488])

In [28]:
np.stack(best_seqsignet_attention_encoder_umap_kfold_20["recall_scores"]).mean(axis=0)

array([0.77164064, 0.42491749, 0.40666667])

# history_length=35

In [29]:
shift = 3
window_size = 5
n = 11

In [ ]:
(
    seqsignet_attention_encoder_umap_kfold_35,
    best_seqsignet_attention_encoder_umap_kfold_35,
    _,
    __,
) = seqsignet_attention_encoder_hyperparameter_search(
    shift=shift,
    window_size=window_size,
    n=n,
    dim_reduce_methods=["umap"],
    results_output=f"{output_dir}/seqsignet_attention_encoder_umap_focal_{gamma}_{shift}_{window_size}_{n}_kfold.csv",
    **kwargs,
)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


##################################################
dimension: 15 | method: umap
given shift 3, window size 5 and n 11: history length = 35
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.


...

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
seqsignet_attention_encoder_umap_kfold_35

In [ ]:
best_seqsignet_attention_encoder_umap_kfold_35["f1"].mean()

In [ ]:
best_seqsignet_attention_encoder_umap_kfold_35["precision"].mean()

In [ ]:
best_seqsignet_attention_encoder_umap_kfold_35["recall"].mean()

In [ ]:
np.stack(best_seqsignet_attention_encoder_umap_kfold_35["f1_scores"]).mean(axis=0)

In [ ]:
np.stack(best_seqsignet_attention_encoder_umap_kfold_35["precision_scores"]).mean(
    axis=0
)

In [ ]:
np.stack(best_seqsignet_attention_encoder_umap_kfold_35["recall_scores"]).mean(axis=0)

# history_length=80

In [ ]:
shift = 3
window_size = 5
n = 26

In [ ]:
(
    seqsignet_attention_encoder_umap_kfold_80,
    best_seqsignet_attention_encoder_umap_kfold_80,
    _,
    __,
) = seqsignet_attention_encoder_hyperparameter_search(
    shift=shift,
    window_size=window_size,
    n=n,
    dim_reduce_methods=["umap"],
    results_output=f"{output_dir}/seqsignet_attention_encoder_umap_focal_{gamma}_{shift}_{window_size}_{n}_kfold.csv",
    **kwargs,
)

In [ ]:
seqsignet_attention_encoder_umap_kfold_80

In [ ]:
best_seqsignet_attention_encoder_umap_kfold_80["f1"].mean()

In [ ]:
best_seqsignet_attention_encoder_umap_kfold_80["precision"].mean()

In [ ]:
best_seqsignet_attention_encoder_umap_kfold_80["recall"].mean()

In [ ]:
np.stack(best_seqsignet_attention_encoder_umap_kfold_80["f1_scores"]).mean(axis=0)

In [ ]:
np.stack(best_seqsignet_attention_encoder_umap_kfold_80["precision_scores"]).mean(
    axis=0
)

In [ ]:
np.stack(best_seqsignet_attention_encoder_umap_kfold_80["recall_scores"]).mean(axis=0)

# history_length=110

In [ ]:
shift = 3
window_size = 5
n = 36

In [ ]:
(
    seqsignet_attention_encoder_umap_kfold_110,
    best_seqsignet_attention_encoder_umap_kfold_110,
    _,
    __,
) = seqsignet_attention_encoder_hyperparameter_search(
    shift=shift,
    window_size=window_size,
    n=n,
    dim_reduce_methods=["umap"],
    results_output=f"{output_dir}/seqsignet_attention_encoder_umap_focal_{gamma}_{shift}_{window_size}_{n}_kfold.csv",
    **kwargs,
)

In [ ]:
seqsignet_attention_encoder_umap_kfold_110

In [ ]:
best_seqsignet_attention_encoder_umap_kfold_110["f1"].mean()

In [ ]:
best_seqsignet_attention_encoder_umap_kfold_110["precision"].mean()

In [ ]:
best_seqsignet_attention_encoder_umap_kfold_110["recall"].mean()

In [ ]:
np.stack(best_seqsignet_attention_encoder_umap_kfold_110["f1_scores"]).mean(axis=0)

In [ ]:
np.stack(best_seqsignet_attention_encoder_umap_kfold_110["precision_scores"]).mean(
    axis=0
)

In [ ]:
np.stack(best_seqsignet_attention_encoder_umap_kfold_110["recall_scores"]).mean(axis=0)